In [1]:
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow as tf

In [2]:
data=tfds.load(name="imdb_reviews",split=["train[:60%]","train[60%:]","test"],as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteORILPL/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteORILPL/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteORILPL/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
train_data,val_data,test_data=data

In [4]:
len(train_data),len(val_data),len(test_data)

(15000, 10000, 25000)

In [5]:
x=train_data.batch(100)

In [6]:
len(x)

150

In [7]:
x_inputs,labels=next(iter(x))

In [8]:
path = "https://tfhub.dev/google/universal-sentence-encoder/4"
hub = hub.KerasLayer(path, input_shape=[], dtype=tf.string, trainable=True)


In [9]:


model = tf.keras.Sequential([
    hub,
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               256797824 
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 256,832,769
Trainable params: 256,832,769


In [10]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=5)
history = model.fit(train_data.shuffle(10000).batch(512),epochs=25,validation_data=val_data.shuffle(10000).batch(512), callbacks=[callback])

Epoch 1/25
30/30 [==============================] - 86s 2s/step - loss: 0.5922 - accuracy: 0.6991 - val_loss: 0.4055 - val_accuracy: 0.8373
Epoch 2/25
30/30 [==============================] - 76s 3s/step - loss: 0.3074 - accuracy: 0.8913 - val_loss: 0.2877 - val_accuracy: 0.8831
Epoch 3/25
30/30 [==============================] - 75s 3s/step - loss: 0.1118 - accuracy: 0.9739 - val_loss: 0.3808 - val_accuracy: 0.8811
Epoch 4/25
30/30 [==============================] - 76s 3s/step - loss: 0.0338 - accuracy: 0.9952 - val_loss: 0.4867 - val_accuracy: 0.8810
Epoch 5/25
30/30 [==============================] - 74s 2s/step - loss: 0.0114 - accuracy: 0.9987 - val_loss: 0.5330 - val_accuracy: 0.8897
Epoch 6/25
30/30 [==============================] - 73s 2s/step - loss: 0.0045 - accuracy: 0.9996 - val_loss: 0.5751 - val_accuracy: 0.8934
Epoch 7/25
30/30 [==============================] - 72s 2s/step - loss: 0.0029 - accuracy: 0.9999 - val_loss: 0.6243 - val_accuracy: 0.8932
Epoch 8/25
30/30 [==

In [11]:
result=model.evaluate(test_data.batch(512))
for name,val in zip(model.metrics_names,result):
    print("%s: %.5f"% (name,val))

49/49 [==============================] - 25s 513ms/step - loss: 0.8589 - accuracy: 0.8846
loss: 0.85886
accuracy: 0.88464


In [ ]:
history = model.fit(train_images, train_labels, epochs=10, batch_size=128, validation_data=(test_images, test_labels))
